In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

In [35]:
# Functions

# reverse the data of argument
def reverseData(data):
    data = data[::-1]
    return data

def getRound(data, decimals = 3):
    return np.around(data, decimals)

## given two curve a, b, then return the inversection point of two curves
def FindTheBuyPoint(a, b, mode):
    if(mode == "rsi_1"):
        buyPoint = np.argwhere(np.diff(np.sign(a - 20)) > 0).reshape(-1) + 0
    else:
        buyPoint = np.argwhere(np.diff(np.sign(a - b)) > 0).reshape(-1) + 0
    return buyPoint

def FindTheSellPoint(a, b, mode):
    if(mode == "rsi_1"):
        sellPoint = np.argwhere(np.diff(np.sign(a - 80)) < 0).reshape(-1) + 0
    else:
        sellPoint = np.argwhere(np.diff(np.sign(a - b)) < 0).reshape(-1) + 0
    return sellPoint

def GetDataFromDB(ip, userName, psw, tableName, sql):
    try:
        db = pymysql.connect(ip, userName, psw, tableName)
    except:
        print ("Error: unable to connect to DB")
        
    #create a cursor
    cursor = db.cursor()
    results = []
    try:
        cursor.execute(sql)
        results = cursor.fetchall()
    except:
        print ("Error: unable to fetch data from DB")
    
    return results
    db.close()

def SMA(data, period):
    if (type(data) == pd.core.series.Series):
        return data.rolling(window = period).mean()
    else:
        print ("Error: Wrong input, SMA(pandas.core.series.Series, integer)")
        
def EMA(data, period):
    if (type(data) == pd.core.series.Series):
        return data.ewm(span = period, adjust = False).mean()
    else:
        print ("Error: Wrong input, EMA(pandas.core.series.Series, integer)")
        
def RSI(data, period):
    if (type(data) == pd.core.series.Series):
        delta = data.diff()
        up, down = delta.copy(), delta.copy()

        up[up < 0] = 0
        down[down > 0] = 0
        down = abs(down)
        
        Up = SMA(up, period)
        Up = Up.fillna(0)
        for i in range(period,len(Up)):
            Up[i] = getRound((Up[i - 1] * (period - 1) + up[i]) / period)

        Down = SMA(down, period)
        Down = Down.fillna(0)
        for i in range(period,len(Down)):
            Down[i] = getRound((Down[i - 1] * (period - 1) + down[i]) / period)


        rsi = 100 * (Up / (Up + Down))
        rsi = rsi.fillna(0)
        return getRound(rsi, 2)
    else:
        print ("Error: Wrong input, RSI(pandas.core.series.Series, integer)")
        
def MACD(data, period = []):
    if (type(data) == pd.core.frame.DataFrame and len(period) == 3):   
        di = (data['High'] + data['Low'] + 2.0 * data['Close']) / 4.0
        ema12 = SMA(di, period[0])
        ema12 = ema12.fillna(0)
        for i in range(period[0] + 1, len(ema12)):
            ema12[i] = (ema12[i - 1] * (period[0] - 1) + di[i] * 2.0) / (period[0] + 1)
    
        ema26 = SMA(di, period[1])
        ema26 = ema26.fillna(0)
        for i in range(period[1] + 1, len(ema26)):
            ema26[i] = (ema26[i - 1] * (period[1] - 1) + di[i] * 2.0) / (period[1] + 1)
    
        dif = ema12 - ema26

        dem = SMA(dif, period[2])
        dem = dem.fillna(0)
        for i in range(period[2] + 1, len(dem)):
            dem[i] = (dem[i - 1] * (period[2] - 1) + dif[i] * 2.0) / (period[2] + 1)
        return {'dif':dif, 'dem':dem}
    else:
        if(type(data) == pd.core.frame.DataFrame):
            print ("Error: Wrong input, MACD(pandas.core.frame.DataFrame, list of integer)")
        elif(len(period) == 3):
            print ("Error: number of content in list do not equal to 3")
            
def KD(data, result = {}):
    close = data['收盤價(元)'].copy()
    for i in range(0,9):
        close[i] = 0

    data['RSV'] = (( data['收盤價(元)'] - data['最低價(元)'].rolling(window = 9).min()) / (data['最高價(元)'].rolling(window = 9).max() - data['最低價(元)'].rolling(window = 9).min()))
    data['RSV'] = data['RSV'].fillna(0)
    if(not result):
        result = {
            'K9':[0],
            'D9' :[0]
        }
    #calculate everyday's KD
    for i in range(1, len(data.index)):
        K9_value = (1.0/3.0) * data['RSV'][i] + (2.0 / 3.0) * result['K9'][i - 1]
        result['K9'].append(getRound(K9_value, 5))
        D9_value = (2.0/3.0) * result['D9'][i - 1] + (1.0 / 3.0) * result['K9'][i]
        result['D9'].append(getRound(D9_value, 5))
        
    return result

In [4]:
# get data
etfPriceData = pd.read_csv(r"C:\Users\user\Desktop\TBrain_DataSet\taetfp.csv", encoding="big5")

In [20]:
# kd
KD_val = pd.DataFrame(KD(etfPriceData))
# kd's long points & short points
KD_long = FindTheBuyPoint(KD_val['K9'], KD_val['D9'], "kd")
KD_short = FindTheSellPoint(KD_val['K9'], KD_val['D9'], "kd")

# create a long/short points list
KD_result = [0 for i in range(len(KD_val))]

for i in KD_long:
    KD_result[i] = 1
for i in KD_short:
    KD_result[i] = -1
for i in range(len(KD_result)):
    if KD_result[i] == 0:
        if abs(KD_val['K9'][i]) < abs(KD_val['D9'][i]):
            KD_result[i] = KD_val['K9'][i] / KD_val['D9'][i]
        else:
            KD_result[i] = KD_val['D9'][i] / KD_val['K9'][i]
KD_result = pd.Series(KD_result).fillna(0).tolist()

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


In [49]:
# rsi
rsi5_result = pd.DataFrame(RSI(etfPriceData['收盤價(元)'], 5))
rsi5_result = rsi5_result.rename(columns={'收盤價(元)': 'rsi'})

rsi14_result = pd.DataFrame(RSI(etfPriceData['收盤價(元)'], 14))
rsi14_result = rsi14_result.rename(columns={'收盤價(元)': 'rsi'})

# rsi's long points & short points
rsi_long_with1curve = FindTheBuyPoint(rsi14_result['rsi'], [], "rsi_1")
rsi_short_with1curve = FindTheSellPoint(rsi14_result['rsi'], [], "rsi_1")

rsi_long_with2curve = FindTheBuyPoint(rsi5_result['rsi'], rsi14_result['rsi'], "rsi_2")
rsi_short_with2curve = FindTheSellPoint(rsi5_result['rsi'], rsi14_result['rsi'], "rsi_2")

# create a long/short points list
rsi1curve_result = [0 for i in range(len(rsi14_result))]
rsi2curve_result = [0 for i in range(len(rsi14_result))]

for i in rsi_long_with1curve:
    rsi1curve_result[i] = 1
for i in rsi_short_with1curve:
    rsi1curve_result[i] = -1

for i in range(len(rsi1curve_result)):
    if rsi1curve_result[i] == 0:
        rsi1curve_result[i] = -1.0 + 2.0 * (rsi14_result['rsi'][i] - 20.0) / (80.0 - 20.0)

for i in rsi_long_with2curve:
    rsi2curve_result[i] = 1
for i in rsi_short_with2curve:
    rsi2curve_result[i] = -1
for i in range(len(rsi2curve_result)):
    if rsi2curve_result[i] == 0:
        if abs(rsi5_result['rsi'][i]) < abs(rsi14_result['rsi'][i]):
            rsi2curve_result[i] = rsi5_result['rsi'][i] / rsi14_result['rsi'][i]
        else:
            rsi2curve_result[i] = rsi14_result['rsi'][i] / rsi5_result['rsi'][i]

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars


In [50]:
for i in range(len(rsi1curve_result)):
    print (rsi14_result['rsi'][i], rsi1curve_result[i])

0.0 -1.6666666666666665
0.0 -1.6666666666666665
0.0 -1.6666666666666665
0.0 -1.6666666666666665
0.0 -1.6666666666666665
0.0 -1.6666666666666665
0.0 -1.6666666666666665
0.0 -1.6666666666666665
0.0 -1.6666666666666665
0.0 -1.6666666666666665
0.0 -1.6666666666666665
0.0 -1.6666666666666665
0.0 -1.6666666666666665
0.0 1
100.0 -1
66.67 0.5556666666666668
24.14 -0.862
15.38 1
40.38 -0.32066666666666654
69.15 0.6383333333333334
71.28 0.7093333333333334
71.26 0.7086666666666668
75.27 0.8423333333333332
81.08 -1
72.41 0.7469999999999999
77.27 0.9089999999999998
79.26 0.9753333333333336
79.84 0.9946666666666668
83.67 -1
70.81 0.6936666666666667
65.43 0.5143333333333335
69.23 0.6410000000000002
66.46 0.5486666666666664
53.44 0.1146666666666667
54.95 0.16500000000000004
58.01 0.2669999999999999
46.89 -0.10366666666666668
53.6 0.1200000000000001
56.76 0.22533333333333316
55.98 0.19933333333333314
61.19 0.373
62.2 0.40666666666666673
56.02 0.20066666666666677
57.49 0.2496666666666667
56.41 0.2136666

52.63 0.08766666666666678
51.07 0.035666666666666735
48.9 -0.036666666666666736
51.14 0.038000000000000034
57.23 0.24099999999999988
61.83 0.3943333333333332
57.06 0.2353333333333334
56.43 0.21433333333333326
60.06 0.3353333333333335
60.4 0.3466666666666667
61.99 0.39966666666666684
64.24 0.47466666666666657
70.28 0.6759999999999999
71.38 0.7126666666666666
72.88 0.7626666666666666
76.88 0.8959999999999999
69.79 0.6596666666666668
68.42 0.6140000000000001
69.35 0.6449999999999998
63.29 0.44300000000000006
64.47 0.4823333333333333
65.03 0.5010000000000001
61.79 0.393
62.36 0.4119999999999999
50.84 0.028000000000000025
53.26 0.10866666666666669
47.84 -0.07199999999999984
48.41 -0.05300000000000016
52.11 0.07033333333333336
56.7 0.22333333333333338
47.08 -0.09733333333333338
46.56 -0.11466666666666658
52.37 0.07899999999999996
57.45 0.24833333333333352
55.52 0.18400000000000016
63.61 0.45366666666666666
64.94 0.498
64.42 0.4806666666666668
70.65 0.6883333333333335
61.95 0.3983333333333334

45.05 -0.16500000000000015
45.26 -0.15800000000000003
44.23 -0.19233333333333347
46.64 -0.11199999999999999
54.71 0.15700000000000003
53.64 0.1213333333333333
56.76 0.22533333333333316
56.67 0.2223333333333335
52.28 0.07600000000000007
54.08 0.1359999999999999
59.59 0.31966666666666677
61.51 0.3836666666666666
56.9 0.22999999999999998
56.76 0.22533333333333316
49.16 -0.028000000000000136
45.23 -0.15900000000000014
44.69 -0.17700000000000005
48.21 -0.059666666666666646
59.7 0.3233333333333335
51.92 0.06400000000000006
54.45 0.14833333333333343
55.6 0.18666666666666676
57.85 0.2616666666666667
55.78 0.19266666666666676
57.96 0.2653333333333334
60.66 0.3553333333333333
61.47 0.3823333333333332
61.4 0.3799999999999999
60.29 0.34299999999999997
61.73 0.3909999999999998
67.89 0.5963333333333334
60.09 0.3363333333333334
58.53 0.2843333333333333
65.0 0.5
63.04 0.43466666666666653
61.36 0.3786666666666667
60.98 0.3659999999999999
55.5 0.18333333333333335
52.17 0.07233333333333336
52.58 0.085999

64.04 0.4680000000000002
64.4 0.4800000000000002
64.04 0.4680000000000002
67.58 0.5859999999999999
50.22 0.007333333333333192
57.83 0.2609999999999999
59.17 0.30566666666666675
55.7 0.19000000000000017
57.94 0.2646666666666666
62.7 0.42333333333333334
56.35 0.21166666666666667
60.62 0.35399999999999987
51.41 0.04699999999999993
54.12 0.1373333333333333
57.35 0.2450000000000001
56.23 0.20766666666666667
57.31 0.2436666666666667
61.69 0.3896666666666666
55.76 0.19199999999999995
60.14 0.3380000000000001
64.58 0.486
77.7 0.9233333333333333
72.58 0.7526666666666666
71.97 0.7323333333333333
57.99 0.2663333333333333
58.02 0.2673333333333334
58.02 0.2673333333333334
61.75 0.3916666666666666
52.64 0.08800000000000008
53.3 0.10999999999999988
54.96 0.16533333333333333
60.29 0.34299999999999997
56.9 0.22999999999999998
53.5 0.1166666666666667
53.62 0.12066666666666648
56.13 0.20433333333333348
54.73 0.15766666666666662
58.24 0.27466666666666684
59.16 0.30533333333333323
59.22 0.30733333333333324

46.47 -0.1176666666666667
48.69 -0.04366666666666674
47.26 -0.09133333333333338
47.27 -0.09099999999999986
47.82 -0.07266666666666666
46.92 -0.10266666666666657
45.58 -0.14733333333333343
37.8 -0.40666666666666673
38.36 -0.388
43.29 -0.22366666666666668
49.64 -0.01200000000000001
46.91 -0.10300000000000009
50.12 0.0040000000000000036
51.94 0.06466666666666665
45.61 -0.14633333333333332
48.51 -0.04966666666666675
45.82 -0.1393333333333333
49.75 -0.008333333333333304
44.2 -0.19333333333333325
43.04 -0.23199999999999998
6.02 -1.466
6.01 -1.4663333333333333
5.97 -1.4676666666666667
5.93 -1.469
5.92 -1.4693333333333334
6.19 -1.4603333333333333
6.25 -1.4583333333333333
6.64 -1.4453333333333334
6.99 -1.4336666666666666
6.97 -1.4343333333333335
7.61 -1.413
7.43 -1.419
8.03 -1.399
7.93 -1.4023333333333334
9.46 -1.3513333333333333
10.58 -1.314
10.49 -1.317
10.33 -1.3223333333333334
11.49 -1.2836666666666667
14.43 -1.1856666666666666
14.66 -1.178
14.67 -1.1776666666666666
15.36 -1.154666666666666

...

70.26 0.6753333333333336
66.15 0.5383333333333336
64.84 0.4946666666666668
66.85 0.5616666666666665
63.22 0.44066666666666654
63.35 0.44500000000000006
65.41 0.5136666666666665
66.0 0.5333333333333334
68.87 0.6290000000000002
70.07 0.6689999999999998
68.57 0.6189999999999998
65.44 0.5146666666666666
61.03 0.3676666666666668
57.04 0.23466666666666658
50.0 0.0
54.11 0.137
46.79 -0.10699999999999998
47.62 -0.07933333333333337
47.45 -0.08499999999999985
51.09 0.03633333333333355
47.79 -0.07366666666666666
47.62 -0.07933333333333337
47.86 -0.07133333333333336
54.03 0.13433333333333342
51.24 0.041333333333333444
51.75 0.05833333333333335
57.85 0.2616666666666667
58.04 0.268
41.67 -0.2776666666666666
40.58 -0.31400000000000006
47.59 -0.08033333333333326
48.92 -0.03599999999999992
44.68 -0.17733333333333334
44.62 -0.17933333333333346
44.63 -0.17899999999999994
45.61 -0.14633333333333332
57.04 0.23466666666666658
61.7 0.3900000000000001
55.1 0.17000000000000015
57.34 0.2446666666666668
63.23 0.

59.14 0.30466666666666664
56.67 0.2223333333333335
58.14 0.2713333333333334
61.71 0.3903333333333334
61.73 0.3909999999999998
58.86 0.2953333333333332
57.33 0.2443333333333333
53.33 0.11099999999999999
61.31 0.377
55.17 0.17233333333333345
56.63 0.22100000000000009
54.38 0.14600000000000013
54.36 0.14533333333333331
56.55 0.21833333333333327
43.68 -0.21066666666666667
40.34 -0.32199999999999984
49.48 -0.017333333333333423
49.44 -0.01866666666666672
43.01 -0.2330000000000001
46.88 -0.10399999999999987
43.75 -0.20833333333333337
43.82 -0.20599999999999996
57.73 0.2576666666666665
57.56 0.252
52.13 0.07100000000000017
58.59 0.28633333333333355
63.14 0.43799999999999994
57.26 0.242
55.41 0.18033333333333323
57.33 0.2443333333333333
64.82 0.4939999999999998
64.68 0.4893333333333336
59.75 0.32499999999999996
57.96 0.2653333333333334
62.55 0.4183333333333332
61.82 0.3939999999999999
61.17 0.3723333333333334
50.65 0.021666666666666723
50.7 0.023333333333333428
53.11 0.10366666666666657
60.09 0

57.22 0.24066666666666658
57.23 0.24099999999999988
61.02 0.3673333333333335
60.98 0.3659999999999999
61.18 0.3726666666666667
67.84 0.5946666666666667
67.92 0.5973333333333335
64.52 0.48399999999999976
64.58 0.486
63.24 0.44133333333333336
53.69 0.123
53.62 0.12066666666666648
63.8 0.45999999999999996
70.11 0.6703333333333332
70.69 0.6896666666666667
70.81 0.6936666666666667
48.62 -0.04600000000000004
48.51 -0.04966666666666675
44.17 -0.19433333333333325
43.98 -0.20066666666666677
56.58 0.21933333333333338
56.6 0.21999999999999997
59.52 0.31733333333333347
66.53 0.5509999999999999
66.67 0.5556666666666668
66.67 0.5556666666666668
62.69 0.4229999999999998
55.19 0.17299999999999982
58.49 0.28300000000000014
59.8 0.32666666666666666
59.79 0.32633333333333336
59.66 0.32199999999999984
60.48 0.3493333333333333
64.74 0.4913333333333332
64.6 0.4866666666666666
60.62 0.35399999999999987
48.65 -0.04500000000000004
47.46 -0.08466666666666667
47.56 -0.08133333333333326
53.22 0.10733333333333328


55.46 0.18199999999999994
44.19 -0.19366666666666676
45.02 -0.16599999999999993
53.96 0.13200000000000012
53.88 0.12933333333333352
54.55 0.15166666666666662
51.46 0.048666666666666636
51.35 0.04500000000000015
38.41 -0.3863333333333334
34.51 -0.5163333333333334
43.83 -0.20566666666666678
45.61 -0.14633333333333332
45.45 -0.15166666666666662
46.95 -0.10166666666666657
49.41 -0.019666666666666832
45.0 -0.16666666666666663
45.04 -0.16533333333333333
44.89 -0.17033333333333334
36.43 -0.45233333333333337
34.66 -0.5113333333333334
28.83 -0.7056666666666667
30.08 -0.664
29.96 -0.6679999999999999
38.55 -0.3816666666666667
38.52 -0.3826666666666666
32.95 -0.5683333333333332
32.4 -0.5866666666666667
36.69 -0.44366666666666676
32.94 -0.5686666666666668
32.91 -0.5696666666666668
30.13 -0.6623333333333334
32.02 -0.5993333333333333
26.05 -0.7983333333333333
31.42 -0.6193333333333333
30.28 -0.6573333333333333
27.73 -0.7423333333333333
26.61 -0.7796666666666667
22.93 -0.9023333333333333
22.18 -0.9273

47.22 -0.09266666666666667
48.55 -0.04833333333333345
50.0 0.0
54.48 0.14933333333333332
57.78 0.2593333333333334
55.81 0.19366666666666665
56.2 0.20666666666666678
50.4 0.013333333333333197
50.0 0.0
56.1 0.20333333333333337
54.24 0.1413333333333333
48.76 -0.041333333333333444
52.85 0.09499999999999997
54.62 0.15399999999999991
56.14 0.20466666666666677
53.64 0.1213333333333333
44.35 -0.18833333333333324
43.22 -0.2260000000000001
51.94 0.06466666666666665
58.27 0.2756666666666667
56.82 0.2273333333333334
62.68 0.42266666666666675
63.43 0.44766666666666666
59.4 0.31333333333333324
60.32 0.3440000000000001
58.2 0.27333333333333343
59.83 0.32766666666666655
47.79 -0.07366666666666666
51.11 0.03699999999999992
17.44 -1.0853333333333333
17.3 -1.09
18.63 -1.0456666666666667
18.67 -1.0443333333333333
18.93 -1.0356666666666667
19.16 1
21.91 -0.9363333333333334
21.7 -0.9433333333333334
21.27 -0.9576666666666667
22.49 -0.917
24.24 -0.8586666666666667
25.27 -0.8243333333333334
29.12 -0.696
28.49 

In [39]:
# sma
sma_result_5 = pd.DataFrame(SMA(etfPriceData['收盤價(元)'], 5))
sma_result_10 = pd.DataFrame(SMA(etfPriceData['收盤價(元)'], 10))
sma_result_5 = sma_result_5.rename(columns={'收盤價(元)':'sma5'})
sma_result_10 = sma_result_10.rename(columns={'收盤價(元)':'sma10'})

sma_result_20 = pd.DataFrame(SMA(etfPriceData['收盤價(元)'], 20))
sma_result_60 = pd.DataFrame(SMA(etfPriceData['收盤價(元)'], 60))
sma_result_20 = sma_result_20.rename(columns={'收盤價(元)':'sma20'})
sma_result_60 = sma_result_60.rename(columns={'收盤價(元)':'sma60'})

sma_result_120 = pd.DataFrame(SMA(etfPriceData['收盤價(元)'], 120))
sma_result_240 = pd.DataFrame(SMA(etfPriceData['收盤價(元)'], 240))
sma_result_120 = sma_result_120.rename(columns={'收盤價(元)':'sma120'})
sma_result_240 = sma_result_240.rename(columns={'收盤價(元)':'sma240'})

sma_result_5.fillna(value=0, inplace=True)
sma_result_10.fillna(value=0, inplace=True)
sma_result_20.fillna(value=0, inplace=True)
sma_result_60.fillna(value=0, inplace=True)
sma_result_120.fillna(value=0, inplace=True)
sma_result_240.fillna(value=0, inplace=True)

# sma's long points & short points
smas_long = FindTheBuyPoint(sma_result_5['sma5'], sma_result_10['sma10'], "sma")
smas_short = FindTheSellPoint(sma_result_5['sma5'], sma_result_10['sma10'], "sma")

smam_long = FindTheBuyPoint(sma_result_20['sma20'], sma_result_60['sma60'], "sma")
smam_short = FindTheSellPoint(sma_result_20['sma20'], sma_result_60['sma60'], "sma")

smal_long = FindTheBuyPoint(sma_result_120['sma120'], sma_result_240['sma240'], "sma")
smal_short = FindTheSellPoint(sma_result_120['sma120'], sma_result_240['sma240'], "sma")

# create a long/short points list
smas_result = [[0] for i in range(len(sma_result_5))]
smam_result = [[0] for i in range(len(sma_result_5))]
smal_result = [[0] for i in range(len(sma_result_5))]

for i in smas_long:
    smas_result[i][0] = 1
for i in smas_short:
    smas_result[i][0] = -1
for i in range(len(smas_result)):
    if smas_result[i][0] == 0:
        if abs(sma_result_5['sma5'][i]) < abs(sma_result_10['sma10'][i]):
            smas_result[i][0] = sma_result_5['sma5'][i] / sma_result_10['sma10'][i]
        else:
            smas_result[i][0] = sma_result_10['sma10'][i] / sma_result_5['sma5'][i]
            
for i in smam_long:
    smam_result[i][0] = 1
for i in smam_short:
    smam_result[i][0] = -1
for i in range(len(smam_result)):
    if smam_result[i][0] == 0:
        if abs(sma_result_20['sma20'][i]) < abs(sma_result_60['sma60'][i]):
            smas_result[i][0] = sma_result_20['sma20'][i] / sma_result_60['sma60'][i]
        else:
            smas_result[i][0] = sma_result_60['sma60'][i] / sma_result_20['sma20'][i]

for i in smal_long:
    smal_result[i][0] = 1
for i in smal_short:
    smal_result[i][0] = -1
for i in range(len(smal_result)):
    if smal_result[i][0] == 0:
        if abs(sma_result_120['sma120'][i]) < abs(sma_result_240['sma240'][i]):
            smas_result[i][0] = sma_result_120['sma120'][i] / sma_result_240['sma240'][i]
        else:
            smas_result[i][0] = sma_result_240['sma240'][i] / sma_result_120['sma120'][i]

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: RuntimeWarning: invalid value encountered in double_scalars


In [11]:
# Draw sma result

In [ ]:
# macd
MACD_val = pd.DataFrame(MACD(etfPriceData, [12,26,9]))
# kd's long points & short points
MACD_long = FindTheBuyPoint(MACD_val['dif'], MACD_val['dem'], "macd")
MACD_short = FindTheSellPoint(MACD_val['dif'], MACD_val['dem'], "macd")

# create a long/short points list
MACD_result = [[0] for i in range(len(MACD_val))]

for i in MACD_long:
    MACD_result[i][0] = 1
for i in MACD_short:
    MACD_result[i][0] = -1
for i in range(len(MACD_result)):
    if MACD_result[i][0] == 0:
        if abs(MACD_val['dif'][i]) < abs(MACD_val['dem'][i]):
            MACD_result[i][0] = MACD_val['dif'][i] / MACD_val['dem'][i]
        else:
            MACD_result[i][0] = MACD_val['dem'][i] / MACD_val['dif'][i]

In [8]:
csvHeader = ['sma_shorterm', 'sma_midterm', 'sma_longterm']
file = open(r"C:\Users\vm06v\OneDrive\桌面\indicator.csv", "w", newline = '')
cursor = csv.writer(file)
cursor.writerow(csvHeader)
for i in range(len(smas_result)):
    cursor.writerow([smas_result[i][0], smam_result[i][0], smal_result[i][0]])